In [0]:
display(dbutils.fs.ls("/mnt/Gen2/"))

path,name,size
dbfs:/mnt/Gen2/Log1.json,Log1.json,211
dbfs:/mnt/Gen2/Log2.json,Log2.json,209
dbfs:/mnt/Gen2/Log3.json,Log3.json,209
dbfs:/mnt/Gen2/Log4.json,Log4.json,209


In [0]:
%fs head /mnt/Gen2/Log4.json

{
 "source": "Application 1",
 "data": "Log data example 1",
 "IP": "10.22.135.7",
 "log_level": "DEBUG",
 "log_type": "Error",
 "log_app": "app",
 "log_timestamp": "2020-08-25T18:45:13+07:00"
}

In [0]:
from pyspark.sql.types import *

logFilePath = "/mnt/Gen2/"

logSchema = StructType([ StructField("source", StringType(), True), StructField("IP", StringType(), True),StructField("log_type", StringType(), True), StructField("log_timestamp", TimestampType(), True) ])


LogDF = (
  spark
    .read
    .schema(logSchema)
    .json(logFilePath)
)

display(LogDF)

source,IP,log_type,log_timestamp
null,null,null,null
null,null,null,null
null,null,null,null
null,null,null,null
null,null,null,null
null,null,null,null
null,null,null,null
null,null,null,null
null,null,null,null
null,null,null,null


In [0]:
from pyspark.sql.functions import *      

logCountDF = (
  LogDF
    .groupBy(col("log_type"))       
    .count()
)
logCountDF.cache()

Out[17]: DataFrame[log_type: string, count: bigint]

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

logFilePath = "/mnt/Gen2/"


logSchema = StructType([ StructField("source", StringType(), True), StructField("IP", StringType(), True),StructField("log_type", StringType(), True), StructField("log_timestamp", TimestampType(), True) ])

streamingLogDF = (
  spark
    .readStream
    .schema(logSchema)              
    .option("maxFilesPerTrigger", 1)
    .json(logFilePath)
)

streamingDataCountDF = (
  streamingLogDF
    .groupBy(col("log_type"))       
    .count()
)

In [0]:
report = (
  streamingDataCountDF
    .writeStream
    .format("memory")        
    .queryName("counts")    
    .outputMode("complete")  
    .start()
)